In [1]:
%matplotlib inline
#!pip install pandas-profiling==2.7.1

In [44]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_profiling as pp
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


In [3]:
def evaluate(model):
    train_preds = model.predict(X_train)
    train_acc = accuracy_score(y_train, train_preds)
    val_preds = model.predict(X_val)
    val_acc = accuracy_score(y_val, val_preds)
    return train_acc, val_acc, train_preds, val_preds
def predict_and_submit(model, fname):
    test_preds = model.predict(df_re_te)
    sub_df['Fault'] = test_preds
    sub_df.to_csv(fname, index=None)
    return sub_df
def radian_conv(degree):
    """
    Return radian.
    """
    return  np.radians(degree)  

In [4]:
df = pd.read_csv('../input/Foml-2021/train.csv', low_memory=False)
print("df.info()")
print(df.info())
df_te = pd.read_csv('../input/Foml-2021/test.csv', low_memory=False)
print("df.info()")
print(df_te.info())

In [5]:
'''
profile = pp.ProfileReport(df, title = 'Is the driver at fault?', html = {'style': {'full_width': True}})
profile.to_notebook_iframe()
profile.to_file(output_file = 'Driver at fault.html')
'''

In [6]:
df_re = df.copy()
df_re_te = df_te.copy()
df_re= df_re.drop(['Local Case Number','Person ID','Vehicle ID', 'Road Name'], axis = 1)
df_re_te= df_re_te.drop(['Local Case Number','Person ID','Vehicle ID', 'Road Name'], axis = 1)

In [7]:
mode = df_re['Vehicle Going Dir'].mode()
df_re.loc[(df['Vehicle Going Dir'] == 'Unknown'),'Vehicle Going Dir']= str(mode)
df_re['Vehicle Going Dir'] =df_re['Vehicle Going Dir'].fillna(str(mode))
mode = df_re['Vehicle Continuing Dir'].mode()
df_re.loc[(df['Vehicle Continuing Dir'] == 'Unknown'),'Vehicle Continuing Dir']= str(mode)
df_re['Vehicle Continuing Dir'] = df_re['Vehicle Continuing Dir'].fillna(str(mode))
mode = df_re['Route Type'].mode()
df_re.loc[(df['Route Type'] == 'Unknown'),'Vehicle Going Dir']= 'North'
df_re['Route Type'] =df_re['Route Type'].fillna(str(mode))


In [8]:
sum(df_re['Route Type'].value_counts())

In [9]:
df_re.isna().sum()

In [10]:
df_re.loc[(df['Agency Name'] == 'Unknown'),'Vehicle Going Dir']= 'North'


In [11]:
df_AgNm = df_re.loc[df['Fault'] == 1]
df_AgNm['Agency Name'].value_counts()

In [12]:
bins = np.linspace(0,5,20)
df_re['Vehicle Going Dir'] = df_re['Vehicle Going Dir'].astype(str)
df_re['Vehicle Continuing Dir'] = df_re['Vehicle Continuing Dir'].astype(str)

plt.hist(df_re['Vehicle Going Dir'], bins, alpha=0.5, label='x')
plt.hist(df_re['Vehicle Continuing Dir'], bins, alpha=0.5, label='y')
plt.legend(loc='upper right')

In [13]:
#df_re= df_re.dropna(subset = ['Local Case Number', 'Route Type','Cross-Street Type','Collision Type', 'Weather', 'Surface Condition', 'Light', 'Traffic Control', 
#                              'Driver Substance Abuse', 'Drivers License State','Vehicle Damage Extent', 'Vehicle First Impact Location',
#                              'Vehicle Second Impact Location', 'Vehicle Body Type', 'Vehicle Movement', 'Vehicle Continuing Dir',
#                              'Vehicle Going Dir','Vehicle Make', 'Vehicle Model','Equipment Problems'] )
df_re['Report Number'].replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)
df_re['Report Number'] = df_re['Report Number'].astype(int)
df_re['Vehicle Year'] = df_re['Vehicle Year'].astype(int)
df_re = df_re.apply(lambda x: x.fillna(str(x.mode())) if x.dtype.kind in 'biufc' else x.fillna('.'))

df_re_te = df_re_te.apply(lambda x: x.fillna(str(x.mode())) if x.dtype.kind in 'biufc' else x.fillna('.'))
df_re_te['Report Number'].replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)
df_re_te['Report Number'] = df_re_te['Report Number'].astype(int)
df_re_te['Vehicle Year'] = df_re_te['Vehicle Year'].astype(int)

In [14]:
half_count = len(df_re) / 2
df_re = df_re.dropna(thresh=half_count,axis=1)
df_re = df_re.apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' else x.fillna('.'))

In [15]:
cols= ['ACRS Report Type', 'Agency Name', 'Route Type','Cross-Street Type', 'Vehicle Going Dir', 'Vehicle Continuing Dir','Weather',
       'Collision Type', 'Surface Condition', 'Light', 'Traffic Control', 'Driver Substance Abuse','Drivers License State', 'Vehicle Damage Extent',
       'Vehicle First Impact Location', 'Vehicle Second Impact Location', 'Vehicle Body Type', 'Vehicle Movement','Vehicle Make', 'Vehicle Model',
       'Equipment Problems','Circumstance','Off-Road Description','Municipality','Related Non-Motorist','Non-Motorist Substance Abuse']
#df_re_te['Local Case Number'].astype(np.float).astype("Int32")

le = preprocessing.LabelEncoder()
df_re[cols]=df_re[cols].apply(le.fit_transform)

le = preprocessing.LabelEncoder()
df_re_te[cols]=df_re_te[cols].apply(le.fit_transform)

In [16]:
'''
for col in cols:
  print(col)
  le = preprocessing.LabelEncoder()
  le.fit(df_re_te[col])
  df_re_te[col] = le.transform(df_re_te[col])
'''

In [17]:
# converting type of columns to 'category'
df_re['Driverless Vehicle'] = df_re['Driverless Vehicle'].astype('category')
# Assigning numerical values and storing in another column
df_re['Driverless Vehicle'] = df_re['Driverless Vehicle'].cat.codes
# converting type of columns to 'category'
df_re['Parked Vehicle'] = df_re['Parked Vehicle'].astype('category')
# Assigning numerical values and storing in another column
df_re['Parked Vehicle'] = df_re['Parked Vehicle'].cat.codes
# converting type of columns to 'category'
df_re['Injury Severity'] = df_re['Injury Severity'].astype('category')
# Assigning numerical values and storing in another column
df_re['Injury Severity'] = df_re['Injury Severity'].cat.codes

# converting type of columns to 'category'
df_re_te['Driverless Vehicle'] = df_re_te['Driverless Vehicle'].astype('category')
# Assigning numerical values and storing in another column
df_re_te['Driverless Vehicle'] = df_re_te['Driverless Vehicle'].cat.codes
# converting type of columns to 'category'
df_re_te['Parked Vehicle'] = df_re_te['Parked Vehicle'].astype('category')
# Assigning numerical values and storing in another column
df_re_te['Parked Vehicle'] = df_re_te['Parked Vehicle'].cat.codes
# converting type of columns to 'category'
df_re_te['Injury Severity'] = df_re_te['Injury Severity'].astype('category')
# Assigning numerical values and storing in another column
df_re_te['Injury Severity'] = df_re_te['Injury Severity'].cat.codes

In [18]:
#print(df_re.info())
sub_df = pd.DataFrame(df_re_te['Id'])
df_re_te = df_re_te.drop('Id', axis =1)

In [19]:
display(df_re.isnull().sum(),
df_re_te.isnull().sum())

In [20]:
df_re['Latitude'] = radian_conv(df_re['Latitude'])
df_re['Longitude'] = radian_conv(df_re['Longitude'])
df_re_te['Latitude']= radian_conv(df_re_te['Latitude'])
df_re_te['Longitude'] = radian_conv(df_re_te['Longitude'])

In [21]:
df_re =df_re.drop('Crash Date/Time', axis = 1)
df_re =df_re.drop('Cross-Street Name', axis = 1)
df_re =df_re.drop('Location', axis = 1)

df_re_te =df_re_te.drop('Crash Date/Time', axis = 1)
df_re_te =df_re_te.drop('Cross-Street Name', axis = 1)
df_re_te =df_re_te.drop('Location', axis = 1)


In [22]:
df_re["Vehicle Year"].value_counts()

In [23]:
display(df_re.shape)
df_re = df_re.loc[df_re["Speed Limit"]>=5]
df_re.shape

df_re = df_re.loc[df_re["Vehicle Year"].between(1995,2016)]
df_re.shape


In [24]:
'''
profile = pp.ProfileReport(df_re, title = 'Is the driver at fault?', html = {'style': {'full_width': True}})
profile.to_notebook_iframe()
profile.to_file(output_file = 'Driver at fault.html')
'''

In [25]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
X_train, X_val, y_train, y_val = train_test_split(df_re.iloc[:, :-1], df_re['Fault'], test_size=0.2, random_state=42)


In [26]:
display(X_train.columns, df_re_te.columns)

In [28]:
DTree = DecisionTreeClassifier(max_depth = 20, min_samples_split = 500, max_features = 6)
DTree.fit(X_train, y_train)
predicted = DTree.predict(X_val)
print('Accuracy: ',accuracy_score(y_val, predicted)*100)
train_acc1,val_acc1,_,_= evaluate(DTree)
print(train_acc1,val_acc1)
predict_and_submit(DTree, 'DTree_finalsubmission.csv')

In [29]:
RF=RandomForestClassifier()
RF.fit(X_train, y_train)
predicted = RF.predict(X_val)
print('Accuracy: ',accuracy_score(y_val, predicted)*100)
train_acc1,val_acc1,_,_= evaluate(RF)
print(train_acc1,val_acc1)
predict_and_submit(RF, 'RF_finalsubmission.csv')

In [ ]:
clf = svm.SVC(C = 0.001, kernel = 'rbf', gamma = 1)
clf.fit(X_train,y_train)
predicted = clf.predict(X_val)
train_acc1,val_acc1,_,_= evaluate(clf)
print(train_acc1,val_acc1)
predict_and_submit(clf, 'svm_finalsubmission.csv')


In [49]:
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [50]:
folds = 10
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='accuracy', n_jobs=4, cv=skf.split(df_re.iloc[:, :-1], df_re['Fault']), verbose=3, random_state=1001 )

random_search.fit(df_re.iloc[:, :-1], df_re['Fault'])


In [53]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best accuracy for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)

In [52]:
predicted = random_search.predict(X_val)
print('Accuracy: ',accuracy_score(y_val, predicted)*100)
train_acc1,val_acc1,_,_= evaluate(random_search)
print(train_acc1,val_acc1)
predict_and_submit(random_search, 'XGB_model_.csv')

In [ ]:
# Tuning hyperparameters to find the best gradient boosting model.  
from sklearn.model_selection import GridSearchCV

para_1 = {'n_estimators':range(20,51,10), 'learning_rate':[0.01,0.1,1,10]}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(),param_grid = para_1, scoring = 'roc_auc',n_jobs=4, cv=5)
gsearch1.fit(X_train, y_train)
gsearch1.best_params_, gsearch1.best_score_
print(gsearch1.best_params_)

para_2 = {'max_depth':range(5,16,5), 'min_samples_leaf':range(30 ,71,10)}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=50, max_features=6, subsample=0.8, random_state=0), param_grid = para_2, scoring='roc_auc',n_jobs=4, cv=5)
gsearch2.fit(X_train, y_train)
gsearch2.best_params_, gsearch2.best_score_
print(gsearch2.best_params_)

In [ ]:
GB_bestmodel = GradientBoostingClassifier(n_estimators=50,learning_rate=1, max_features=6,max_depth=15,min_samples_leaf=60, random_state=0)
GB_bestmodel.fit(X_train, y_train)
train_acc1,val_acc1,_,_= evaluate(GB_bestmodel)
print(train_acc1,val_acc1)
predict_and_submit(GB_bestmodel, 'GB_bestmodel_finalsubmission.csv')